## Final Project Submission

Please fill out:
* Student name: Nastaran Nazemian
* Student pace: part time
* Scheduled project review date/time: ??????????
* Instructor name: Eli Thomas
* Blog post URL:?????????????


In [1]:
from bs4 import BeautifulSoup
import requests 
from datetime import datetime
from datetime import date

In [2]:
def search_movies_page(index):
    html_page = requests.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2020-01-01&countries=us&start=' + str(index * 50 + 1))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    movies = soup.find_all('div', "lister-item-content")
    urls = []
    for i in range(0,len(movies)):
        movies[i]
        movie_url = 'https://www.imdb.com' + movies[i].find('h3', 'lister-item-header').find('a')['href'] 
#         print(movie_url)
        urls.append(movie_url)
    return(urls)

# url = search_movies_page(0)
#url


In [3]:
def movie_page(url):
    movie_page = requests.get(url)
    movie = BeautifulSoup(movie_page.content, 'html.parser')
    return movie

#this_movie_page = movie_page("https://www.imdb.com/title/tt8367814/")
#movie_page(url)

In [4]:
#def movie_id(index,movie_index)    
#     title = {}
#     movie_title_id =search_movies_page(index)[movie_index].split('/')
#     title_id = movie_title_id[-2]
#     title[Title] = title_id 
#return(title)

In [5]:
def movie_id(url):
    title = {}
    movie_title_id = url.split('/')
    title_id = movie_title_id[-2]
    title['Title_id'] = title_id
    return title
#movie_id(url)


In [6]:
def boxoffice(this_movie_page):
    box_office = ['Budget:','Gross USA:','Cumulative Worldwide Gross:','Opening Weekend USA:']
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    for x in box_office:
        finance_value = this_movie_page.find('h4', string= x)
        if finance_value is not None:
            x_money = finance_value.parent.contents[2].replace('$',"").replace(',',"")
            results[x] = x_money
        else:
            results[x] = 'None'
    return results
#boxoffice(url)  

In [7]:
def movie_info(this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    title = this_movie_page.find('div', class_="title_wrapper").find('h1', class_="").contents[0].replace('\xa0','')   
    year = this_movie_page.find('div', class_="title_wrapper").find('a').contents[0]  
    popularity = this_movie_page.find_all('div', class_ = "titleReviewBarSubItem")[-1].find('span', "subText").contents[0].strip().split(' ')[0].strip().replace(',','')
    results = {'Title':title,'Year': year,'Popularity':popularity}
    return results
#movie_info(url)

In [8]:
def movie_genres(this_movie_page):
    #this_movie_page = movie_page(index, movie_index)
    genres_out = {}
    genres_list = this_movie_page.find('div', class_="title_wrapper").find('div', class_="subtext").text.split("|")
    genres = genres_list[2].split()
    genres = list(map(lambda genre: genre.replace(',',""),genres))
    genres_out['Geres'] = genres 
    return genres_out
#movie_genres(url)

In [9]:
def movie_runtime(this_movie_page):
    #this_movie_page = movie_page(index,movie_index)
    runtime_out = {}
    runtime = this_movie_page.find('h4',string="Runtime:").find_next_sibling('time').text.split()
    runtime_min = runtime[0]
    runtime_out['Runtime_min'] = runtime_min
    return runtime_out
#movie_runtime(url)


In [10]:
def movie_release_day(this_movie_page):
    #this_movie_page = movie_page(index,movie_index)
    release_day = {}
    release_day_list = this_movie_page.find('div', class_ = "subtext").find('a', title = "See more release dates").text.split()
    date = " ".join(release_day_list[0:3])
    release_day_format = datetime.strptime(date, '%d %B %Y').strftime("%m/%d/%Y")
    release_day['Release_day'] = release_day_format
    return release_day
#movie_release_day(url)

In [11]:
def rating (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index, movie_index)
    check_rating = this_movie_page.find('div', class_="ratingValue")
    if check_rating is not None:
        rating = check_rating.find('strong')['title'].split()
        imdb_rate = float(rating[0])
        rating_count = rating[3].replace(',','')
        results['IMDB_Rate'] = imdb_rate
        results['Rating_count'] = rating_count
    else:
        results['IMDB_Rate'] = 'None'
        results['Rating_count'] = 'None'
        
    check_value = this_movie_page.find('div',"metacriticScore score_mixed titleReviewBarSubItem")
    if check_value is not None:  
        meta_score = check_value.find('span').text
        results['Meta_Score'] = meta_score
    else:
        results['Meta_Score'] = 'None'
    return results
#rating(url)

In [12]:
def casts_names(this_movie_page):
    roles = ['Director','Writer', 'Star']
    results = {}

            #this_movie_page = movie_page(index,movie_index)
    for role in roles:
        cast_role = (role+':')

        name_list = []
        check_role = this_movie_page.find('h4',string=role+':')
        if (check_role is None):
            check_role = this_movie_page.find('h4',string=role+'s:')
        names = check_role.find_next_siblings('a')
        for name in (names[:3]):
            name_list.append(name.text)
        results[role] = name_list
    return results

# casts_names(this_movie_page)

In [13]:
##Find Number_User_Review, Number_Critical_Review
def review (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    check_value = this_movie_page.find('div', "titleReviewBarItem titleReviewbarItemBorder")
    if check_value is not None:
        review = check_value.find('span', "subText")
        user = review.find('a').text.split()
        number_user_review = user[0].replace(',','')
        results['User_Review_Number'] = number_user_review
        critic = review.find('a', href = "externalreviews").text.split()
        number_critical_review = critic[0]
        results['Critical_Review_Number'] = number_critical_review
    else:
        results['User_Review_Number'] = 'None'
        results['Critical_Review_Number'] = 'None'
    return(results)
#review(url)

In [14]:
##Find Meta_rating,  Genera, and Release_Day---->sub_text(Tird part)
def meta_rate (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    check_value = this_movie_page.find('div', class_="title_wrapper")
    if check_value is not None:
        sub_text =check_value.find('div', class_ = "subtext").text.split('|')
        meta_rating = sub_text[0].strip()
        results['Meta_Rating'] = meta_rating

    else:
        results['Meta_Rating'] = 'None'
    
    return(results)

#meta_rate(url)

In [15]:
# this_movie_page = movie_page(url)
def one_movie_info(url):
    this_movie_page = movie_page(url)
    movie_data = {}
    movie_function_list = [movie_id(url), 
                           boxoffice(this_movie_page),
                           movie_info(this_movie_page),
                           movie_genres(this_movie_page),
                           movie_runtime(this_movie_page),
                           movie_release_day(this_movie_page),
                           rating (this_movie_page),
                           casts_names(this_movie_page),
                           review (this_movie_page),
                           meta_rate (this_movie_page)]
    for function in movie_function_list:
        movie_data.update(function)
        
        
#     movie_data.update(movie_id(this_movie_page))
    
    return(movie_data)
# len(one_movie_info(url))  


In [16]:
# urls = search_movies_page(0)
# #print(len(urls))
# movies_data = []
# for url in urls:
# #     this_movie_page = movie_page(url)
#     one_row = one_movie_info(url)
#     movies_data.append(one_row)
#     #print(len(movies_data))
# print(len(movies_data))

In [ ]:
movies_data = []
for index in range (1,2):
    urls = search_movies_page(index)
#     print(len(urls))
    for url in urls:
        print(url)
        one_row = one_movie_info(url)
        movies_data.append(one_row)
#         print(len(movies_data))
print('end', len(movies_data))

https://www.imdb.com/title/tt7984766/
https://www.imdb.com/title/tt0437086/
https://www.imdb.com/title/tt1489887/
https://www.imdb.com/title/tt6450804/
https://www.imdb.com/title/tt4332232/
https://www.imdb.com/title/tt5814534/
https://www.imdb.com/title/tt8305806/
https://www.imdb.com/title/tt6513120/
https://www.imdb.com/title/tt7879442/
https://www.imdb.com/title/tt8663516/
https://www.imdb.com/title/tt2935510/
https://www.imdb.com/title/tt7798634/
https://www.imdb.com/title/tt3224458/
https://www.imdb.com/title/tt9071322/
https://www.imdb.com/title/tt7456310/
https://www.imdb.com/title/tt4364194/
https://www.imdb.com/title/tt6521876/
https://www.imdb.com/title/tt0448115/
https://www.imdb.com/title/tt5073642/
https://www.imdb.com/title/tt9243946/
https://www.imdb.com/title/tt8299768/


In [ ]:
import re
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.DataFrame(movies_data)
df.head()


In [ ]:
df.describe()

In [ ]:
df.info()
df.shape
